# Data preprocessing

## Important functions to be used multiple times

In [ ]:
# Calculate the z-score for a specified column
def get_z_scores(column_name):
    z_scores = np.abs((df[column_name] - df[column_name].mean()) / df[column_name].std())
    return z_scores

# Remove outliers from the DataFrame based on z-scores
def remove_outliers(z_scores):
    # Define a threshold for outlier detection (e.g., z-score > 3)
    threshold = 3

    # Filter rows where z-score exceeds the threshold
    outliers = df[z_scores > threshold]

    # Remove outliers from the DataFrame
    cleaned_df = df[z_scores <= threshold]
    return cleaned_df

## HS_Cigar_Use

In [9]:
import pandas as pd
import numpy as np
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()

# Read data from the MySQL table into a pandas DataFrame
query = "SELECT * FROM cdc_cigar_use"
df = pd.read_sql(query, conn)

# Close the MySQL connection
conn.close()

# Specify the columns with potential outliers
dirty_columns = [
    'Greater_Risk_Data_Value',
    'Greater_Risk_Low_Confidence_Limit',
    'Greater_Risk_High_Confidence_Limit',
    'Lesser_Risk_Data_Value',
    'Lesser_Risk_Low_Confidence_Limit',
    'Lesser_Risk_High_Confidence_Limit'
]

# Iterate through the columns to remove outliers
for column in dirty_columns:
    z_scores = get_z_scores(column)
    cleaned_df = remove_outliers(z_scores)

print("Outliers removed and inserted successfully!")


Outliers removed and inserted successfully!


/var/folders/9q/yldjq0js2ng8cysrh56tt9xw0000gn/T/ipykernel_38943/708248302.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [10]:
conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()

# Truncate the existing table to remove all data
truncate_query = "TRUNCATE TABLE cdc_cigar_use"
cursor.execute(truncate_query)
conn.commit()

# Insert the cleaned data into the MySQL table
insert_query = (
    "INSERT INTO cdc_cigar_use "
    "(YEAR, LocationAbbr, LocationDesc, DataSource, Topic, Subtopic, "
    "ShortQuestionText, Greater_Risk_Question, Description, Data_Value_Symbol, "
    "Data_Value_Type, Greater_Risk_Data_Value, Greater_Risk_Data_Value_Footnote_Symbol, "
    "Greater_Risk_Data_Value_Footnote, Greater_Risk_Low_Confidence_Limit, "
    "Greater_Risk_High_Confidence_Limit, Lesser_Risk_Question, Lesser_Risk_Data_Value, "
    "Lesser_Risk_Data_Value_Footnote_Symbol, Lesser_Risk_Data_Value_Footnote, "
    "Lesser_Risk_Low_Confidence_Limit, Lesser_Risk_High_Confidence_Limit, Sample_Size, "
    "Sex, Race, Grade, SexualIdentity, SexOfSexualContacts, GeoLocation, TopicId, "
    "SubTopicID, QuestionCode, LocationId, StratID1, StratID2, StratID3, StratID4, "
    "StratID5, StratificationType, StratID6) "
    "VALUES "
    "(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, "
    "%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)

for row in cleaned_df.itertuples(index=False):
    cursor.execute(insert_query, row)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Data Cleaned and inserted successfully!")


Cleaned data inserted successfully!
